# Queries to use for A3c, A3d, A3e, A3f


In [1]:
# Required libraries
import os
import time
import boto3
import logging
import pandas as pd
from typing import Dict

In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# configuration details 
SCHEMA_NAME = "schema_name"

# fill in your net id below.
netid = "imb59"
S3_STAGING_PREFIX = "data/a05"
S3_BUCKET_NAME = f"athena-{netid}"
S3_STAGING_DIR = f"s3://{S3_BUCKET_NAME}/{S3_STAGING_PREFIX}/"
S3_OUTPUT_DIRECTORY = "data"
AWS_REGION = "us-east-1"

In [4]:
# initialize the Athena client
athena_client = boto3.client("athena", region_name=AWS_REGION)

In [5]:
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    logger.info("download_and_load_query_results, enter")
    while True:
        try:
            # This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.5)
            else:
                raise err
    logger.info(f"Time to complete query: {time.time() - start_time}s")
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        region_name=AWS_REGION,
    )
    s3_path = os.path.join(S3_STAGING_PREFIX, f"{query_response['QueryExecutionId']}.csv")
    logger.info(f"downloading file from S3_BUCKET_NAME={S3_BUCKET_NAME}, s3_path={s3_path}, to local file {temp_file_location}")
    s3_client.download_file(
        S3_BUCKET_NAME,
        s3_path,
        temp_file_location,
    )
    df = pd.read_csv(temp_file_location)
    logger.info(f"results dataframe shape is {df.shape}")
    return df

## Sorting Subreddits Relevant to: 

(A1, B3a, B13), A2b, A3a, A3b

* “Frustrating” or “frustrat” and “cancer” (HINTS A2b)
* “cancer” and “doctors” or “trust” (i.e. does not “need” to contain trust because trust is included in the NRC sentiment analysis) (HINTS A3a)
* “cancer” and “family” or “friends” or “sister” or “brother” or “mother” or “mom” or “father” or “mother” or “cousin” or “aunt” or “uncle” or “trust” (HINTS A3b)

HINTS Questions: 

* SeekCancerInfo: A1 | Have you ever looked for information about cancer from any source?
* Electronic2_HealthInfo: B3a | In the past 12 months have you used the Internet to look for health or medical information?
* MisleadingHealthInfo: B13 | How much of the health information that you see on social media do you think is false or misleading?


In [17]:
# query to get comments containing "frustrat" and "cancer"
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit"
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%frustrat%' AND LOWER("body") LIKE '%cancer%'
AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A2b.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())


[2024-11-13 00:56:37,264] p147 {1458311848.py:27} INFO - {'QueryExecutionId': '24db8b2e-8acd-4493-9046-1c0c8d561b2b', 'ResponseMetadata': {'RequestId': 'abfe48e2-09fc-4ae2-acb7-445c1b23286d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 00:56:37 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'abfe48e2-09fc-4ae2-acb7-445c1b23286d'}, 'RetryAttempts': 0}}
[2024-11-13 00:56:37,265] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 00:56:43,069] p147 {272223420.py:17} INFO - Time to complete query: 5.897244453430176s
[2024-11-13 00:56:43,076] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/24db8b2e-8acd-4493-9046-1c0c8d561b2b.csv, to local file athena_query_results.csv
[2024-11-13 00:56:43,148] p147 {272223420.py:31} INFO - results dataframe shape is (9, 4)
[2024-11-13 00:56:43,158] p147 {1458311848.py:33} INFO - Res

                                                body  controversiality  \
0  I'd like to think I've dealt with cancer head ...                 0   
1  thank you for pointing out how it’s misunderst...                 0   
2  It made me realise exactly how little I had. A...                 0   
3  Absolutely! There aren’t many of us with high ...                 0   
4  You are not alone... I am a 22y/o and I had my...                 0   

   created_utc            subreddit  
0   1719826058  CancerFamilySupport  
1   1719799836        CrohnsDisease  
2   1719847236         breastcancer  
3   1719851099        thyroidcancer  
4   1719802666        thyroidcancer  


In [19]:
# Query to get comments containing "cancer" and either "doctors" or "trust"
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit"
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (LOWER("body") LIKE '%doctors%' OR LOWER("body") LIKE '%trust%') AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3a.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())


[2024-11-13 00:58:49,157] p147 {586797257.py:27} INFO - {'QueryExecutionId': '67a75251-1ea3-4ea0-85e4-14e6a70a96b7', 'ResponseMetadata': {'RequestId': '7fd642a1-ce58-43b2-ba73-12868a7a513e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 00:58:49 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '7fd642a1-ce58-43b2-ba73-12868a7a513e'}, 'RetryAttempts': 0}}
[2024-11-13 00:58:49,158] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 00:58:53,418] p147 {272223420.py:17} INFO - Time to complete query: 4.370722770690918s
[2024-11-13 00:58:53,426] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/67a75251-1ea3-4ea0-85e4-14e6a70a96b7.csv, to local file athena_query_results.csv
[2024-11-13 00:58:53,505] p147 {272223420.py:31} INFO - results dataframe shape is (39, 4)
[2024-11-13 00:58:53,511] p147 {586797257.py:33} INFO - Resu

                                                body  controversiality  \
0  We're sorry to hear that you need to visit thi...                 0   
1  We're sorry to hear that you need to visit thi...                 0   
2  Ohmygosh you poor darling.\n\nBear in mind tha...                 0   
3  I'll go to the ER at literally the first sign ...                 0   
4  Hello! 22f here. I went through almost the sam...                 0   

   created_utc            subreddit  
0   1719792189  doihavebreastcancer  
1   1719799351  doihavebreastcancer  
2   1719813592  CancerFamilySupport  
3   1719819595           Lymphedema  
4   1719824129        thyroidcancer  


In [20]:
# Query to get comments containing "cancer" and any of the specified family-related terms or "trust"
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit"
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (
      LOWER("body") LIKE '%family%' 
      OR LOWER("body") LIKE '%friends%' 
      OR LOWER("body") LIKE '%sister%' 
      OR LOWER("body") LIKE '%brother%' 
      OR LOWER("body") LIKE '%mother%' 
      OR LOWER("body") LIKE '%mom%' 
      OR LOWER("body") LIKE '%father%' 
      OR LOWER("body") LIKE '%cousin%' 
      OR LOWER("body") LIKE '%aunt%' 
      OR LOWER("body") LIKE '%uncle%' 
      OR LOWER("body") LIKE '%trust%'
  ) AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3b.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())


[2024-11-13 01:01:21,213] p147 {3378693322.py:39} INFO - {'QueryExecutionId': 'a2c92059-0889-4d3e-89f4-3c8198a68e9b', 'ResponseMetadata': {'RequestId': '5ea15dc0-6799-4986-b194-55cf7f4037a9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 01:01:21 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '5ea15dc0-6799-4986-b194-55cf7f4037a9'}, 'RetryAttempts': 0}}
[2024-11-13 01:01:21,214] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 01:01:26,525] p147 {272223420.py:17} INFO - Time to complete query: 5.435483932495117s
[2024-11-13 01:01:26,531] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/a2c92059-0889-4d3e-89f4-3c8198a68e9b.csv, to local file athena_query_results.csv
[2024-11-13 01:01:26,614] p147 {272223420.py:31} INFO - results dataframe shape is (84, 4)
[2024-11-13 01:01:26,620] p147 {3378693322.py:45} INFO - Re

                                                body  controversiality  \
0  Thanks for your reply. I went to a v famous br...                 0   
1  I had to comment here, not because I can think...                 0   
2  Alternative therapies are suicide.\n\nI got di...                 0   
3  By no means am I a doctor & I don’t have child...                 0   
4  Just one? “The Real Anthony Fauci reveals how ...                 0   

   created_utc            subreddit  
0   1719826982  doihavebreastcancer  
1   1719828932     CancerCaregivers  
2   1719829111                  cll  
3   1719830928  doihavebreastcancer  
4   1719830990         publichealth  


In [28]:
# merge the datasets based on the first three hints questions 
import pandas as pd
df_query_a2b = pd.read_csv('Query_A2b.csv')
df_query_a3a = pd.read_csv('Query_A3a.csv')
df_query_a3b = pd.read_csv('Query_A3b.csv')

merged_df = pd.concat([df_query_a2b, df_query_a3a, df_query_a3b], ignore_index= True)
merged_df.to_csv('Merged_Data_A2b_A3a_A3b.csv', index = False)

## Sorting Subreddits Relevant to: 

A3c, A3d, A3e, A3f

* “cancer” and government_healthcare_programs = [ "medicare", "medicaid", "children’s health insurance program", "chip", "veterans health administration", "vha", "indian health service", "ihs", "federal employees health benefits program", "fehbp", "affordable care act", "aca", "health insurance marketplace", "public health depart", "local health depart", "national health service corps", "nhsc", "community health centers", "chcs", "national institutes of health", "nih", "nci", "national cancer institute" ] or “trust” (HINTS A3c)

* “cancer” and cancer_charities = [ "american cancer society”, “acs", "cancer research institute", "breast cancer research foundation", "bcrf", "leukemia lymphoma society", "lls", "stand up to cancer", "su2c", "susan g. komen for the cure", "st. jude children’s", "national foundation for cancer research", "nfcr", "livestrong", "mesothelioma research foundation", "prostate cancer foundation", "american brain tumor association", "abta", "colon cancer coalition", "the american institute for cancer research", "aicr" ] or “trust” (HINTS A3d)
* “cancer” and charitable_religious_organizations = [ "catholic relief services”, “crs", "world vision", "samaritan", "jewish federations of north america", "islamic relief worldwide", "buddhist global relief", "the salvation army", "christian aid", "lutheran world relief", "tzu chi foundation", "care”, “cooperative for assistance and relief everywhere", "habitat for humanity", "church world service”, “cws", "heifer international" ] or “trust” (HINTS A3e)
* “cancer” and top_cancer_institutes = [ "researcher", "scientist", "physicians", "md anderson cancer center", "memorial sloan kettering cancer center", "msk", "mayo clinic cancer center", "johns hopkins sidney kimmel comprehensive cancer center", "cleveland clinic", "ucla medical center", "massachusetts general hospital cancer center", "duke cancer institute", "stanford cancer institute", "university of california, san francisco medical center", "ucsf", "northwestern medicine feinberg school of medicine", "university of pennsylvania abramson cancer center", "roswell park comprehensive cancer center", "fred hutchinson cancer research center" ] or “trust” (HINTS A3f)


HINTS Questions: 

* CancerTrustGov: A3c. In general, how much would you trust information about cancer from government health agencies?
* CancerTrustCharities: A3d. In general, how much would you trust information about cancer from charitable organizations?
* CancerTrustReligiousOrgs: A3e. In general, how much would you trust information about cancer from religious organizations and leaders?
* CancerTrustScientists: A3f. In general how much would you trust information about cancer from scientists?.

In [21]:
# Query to get comments containing "cancer" and nay goverment healthcare programs 
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit" 
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (
      LOWER("body") LIKE '%medicare%' 
      OR LOWER("body") LIKE '%medicaid%' 
      OR LOWER("body") LIKE '%"children’s health insurance program%' 
      OR LOWER("body") LIKE '%chip%' 
      OR LOWER("body") LIKE '%veterans health administration%' 
      OR LOWER("body") LIKE '%vha%' 
      OR LOWER("body") LIKE '%indian health service%' 
      OR LOWER("body") LIKE '%ihs%' 
      OR LOWER("body") LIKE '%federal employees health benefits program%' 
      OR LOWER("body") LIKE '%fehbp%' 
      OR LOWER("body") LIKE '%affordable care act%'
      OR LOWER("body") LIKE '%aca%'
      OR LOWER("body") LIKE '%health insurance marketplace%'
      OR LOWER("body") LIKE '%public health depart%'
      OR LOWER("body") LIKE '%local health depart%'
      OR LOWER("body") LIKE '%national health service corps%'
      OR LOWER("body") LIKE '%nhsc%'
      OR LOWER("body") LIKE '%community health centers%'
      OR LOWER("body") LIKE '%national institutes of health%'
      OR LOWER("body") LIKE '%nih%'
      OR LOWER("body") LIKE '%nci%'
      OR LOWER("body") LIKE '%national cancer institute%'
      OR LOWER("body") LIKE '%trust%'
  ) AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3c.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())

[2024-11-13 01:07:03,964] p147 {837093097.py:51} INFO - {'QueryExecutionId': 'a6c6c2c1-2751-4041-bd92-4cae078a0ce1', 'ResponseMetadata': {'RequestId': 'e4d70749-c491-46c1-8689-5f1eb37b8ddb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 01:07:03 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'e4d70749-c491-46c1-8689-5f1eb37b8ddb'}, 'RetryAttempts': 0}}
[2024-11-13 01:07:03,965] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 01:07:11,830] p147 {272223420.py:17} INFO - Time to complete query: 8.003482103347778s
[2024-11-13 01:07:11,836] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/a6c6c2c1-2751-4041-bd92-4cae078a0ce1.csv, to local file athena_query_results.csv
[2024-11-13 01:07:11,911] p147 {272223420.py:31} INFO - results dataframe shape is (31, 4)
[2024-11-13 01:07:11,915] p147 {837093097.py:57} INFO - Resu

                                                body  controversiality  \
0  When I got diagnosed with Breast cancer & I ca...                 0   
1  I'm confused about your self-diagnosis of panc...                 0   
2  Ohmygosh you poor darling.\n\nBear in mind tha...                 0   
3  I'll go to the ER at literally the first sign ...                 0   
4  Hi misskimchigirl, \n\n\nDid your menstrual cy...                 0   

   created_utc            subreddit  
0   1719797834               cancer  
1   1719812763     pancreaticcancer  
2   1719813592  CancerFamilySupport  
3   1719819595           Lymphedema  
4   1719823893         breastcancer  


In [22]:
# Query to get comments containing "cancer" and certain cancer chareties 
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit" 
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (
      LOWER("body") LIKE '%american cancer society%' 
      OR LOWER("body") LIKE '%acs%' 
      OR LOWER("body") LIKE '%cancer research institute%' 
      OR LOWER("body") LIKE '%breast cancer research foundation%' 
      OR LOWER("body") LIKE '%veterans health administration%' 
      OR LOWER("body") LIKE '%bcrf%' 
      OR LOWER("body") LIKE '%leukemia lymphoma society%' 
      OR LOWER("body") LIKE '%ihs%' 
      OR LOWER("body") LIKE '%lls%' 
      OR LOWER("body") LIKE '%stand up to cancer%' 
      OR LOWER("body") LIKE '%su2c%'
      OR LOWER("body") LIKE '%susan g. komen for the cure%'
      OR LOWER("body") LIKE '%st. jude children’s%'
      OR LOWER("body") LIKE '%national foundation for cancer research%'
      OR LOWER("body") LIKE '%nfcr%'
      OR LOWER("body") LIKE '%livestrong%'
      OR LOWER("body") LIKE '%nhsc%'
      OR LOWER("body") LIKE '%mesothelioma research foundation%'
      OR LOWER("body") LIKE '%prostate cancer foundation%'
      OR LOWER("body") LIKE '%american brain tumor association%'
      OR LOWER("body") LIKE '%abta%'
      OR LOWER("body") LIKE '%colon cancer coalition%'
      OR LOWER("body") LIKE '%the american institute for cancer research%'
      OR LOWER("body") LIKE '%aicr%'
      OR LOWER("body") LIKE '%trust%'
  ) AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3d.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())

[2024-11-13 01:08:02,401] p147 {2191114331.py:53} INFO - {'QueryExecutionId': '89cf8f83-037d-4cb3-bf9d-af4fe056c5fb', 'ResponseMetadata': {'RequestId': 'a32f2e6c-3aa7-45f4-8310-84e5975afdbf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 01:08:02 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'a32f2e6c-3aa7-45f4-8310-84e5975afdbf'}, 'RetryAttempts': 0}}
[2024-11-13 01:08:02,402] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 01:08:08,246] p147 {272223420.py:17} INFO - Time to complete query: 5.953483819961548s
[2024-11-13 01:08:08,252] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/89cf8f83-037d-4cb3-bf9d-af4fe056c5fb.csv, to local file athena_query_results.csv
[2024-11-13 01:08:08,325] p147 {272223420.py:31} INFO - results dataframe shape is (36, 4)
[2024-11-13 01:08:08,330] p147 {2191114331.py:59} INFO - Re

                                                body  controversiality  \
0  When I had my surgery I felt the same. This fe...                 0   
1  I did IV, no port. I used ice bags on my hands...                 0   
2  Did they really mean prevent it? What could be...                 0   
3  Today was cousin went through with the decisio...                 0   
4  I have a port. It's my second one. First one I...                 0   

   created_utc            subreddit  
0   1719802120     testicularcancer  
1   1719802713         breastcancer  
2   1719803480    endometrialcancer  
3   1719806973  CancerFamilySupport  
4   1719808898         breastcancer  


In [23]:
# Query to get comments containing "cancer" and certain charitable religious orgs 
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit" 
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (
      LOWER("body") LIKE '%catholic relief services%' 
      OR LOWER("body") LIKE '%crs%' 
      OR LOWER("body") LIKE '%world vision%' 
      OR LOWER("body") LIKE '%samaritan%' 
      OR LOWER("body") LIKE '%jewish federations of north america%' 
      OR LOWER("body") LIKE '%islamic relief worldwide%' 
      OR LOWER("body") LIKE '%buddhist global relief%' 
      OR LOWER("body") LIKE '%the salvation army%' 
      OR LOWER("body") LIKE '%christian aid%' 
      OR LOWER("body") LIKE '%lutheran world relief%' 
      OR LOWER("body") LIKE '%"tzu chi foundation%'
      OR LOWER("body") LIKE '%susan g. komen for the cure%'
      OR LOWER("body") LIKE '%care%'
      OR LOWER("body") LIKE '%cooperative for assistance and relief everywhere%'
      OR LOWER("body") LIKE '%habitat for humanity%'
      OR LOWER("body") LIKE '%church world service%'
      OR LOWER("body") LIKE '%cws%'
      OR LOWER("body") LIKE '%heifer international%'
      OR LOWER("body") LIKE '%trust%'
  ) AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3e.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())

[2024-11-13 01:08:46,728] p147 {2876347991.py:47} INFO - {'QueryExecutionId': '4f5dba25-7985-43f2-a956-adc58e5c8711', 'ResponseMetadata': {'RequestId': '895b6df2-0ed1-4be8-b397-5d5986be96ff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 01:08:46 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '895b6df2-0ed1-4be8-b397-5d5986be96ff'}, 'RetryAttempts': 0}}
[2024-11-13 01:08:46,729] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 01:08:52,094] p147 {272223420.py:17} INFO - Time to complete query: 5.476090431213379s
[2024-11-13 01:08:52,102] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/4f5dba25-7985-43f2-a956-adc58e5c8711.csv, to local file athena_query_results.csv
[2024-11-13 01:08:52,177] p147 {272223420.py:31} INFO - results dataframe shape is (58, 4)
[2024-11-13 01:08:52,183] p147 {2876347991.py:53} INFO - Re

                                                body  controversiality  \
0  I had my abdormal pap about a month before my ...                 0   
1  This subreddit is heavily centered on patients...                 0   
2  It sounds like you are doing fine. You had sur...                 0   
3  Thanks for your reply, and I agree, I apprecia...                 0   
4  I'm going to get downvoted into oblivion for t...                 0   

   created_utc          subreddit  
0   1719795113  PreCervicalCancer  
1   1719795253       breastcancer  
2   1719798254       breastcancer  
3   1719847712                cll  
4   1719849279            nursing  


“cancer” and top_cancer_institutes = [ "researcher", "scientist", "physicians", "md anderson cancer center", "memorial sloan kettering cancer center", "msk", "mayo clinic cancer center", "johns hopkins sidney kimmel comprehensive cancer center", "cleveland clinic", "ucla medical center", "massachusetts general hospital cancer center", "duke cancer institute", "stanford cancer institute", "university of california, san francisco medical center", "ucsf", "northwestern medicine feinberg school of medicine", "university of pennsylvania abramson cancer center", "roswell park comprehensive cancer center", "fred hutchinson cancer research center" ] or “trust” (HINTS A3f)

In [24]:
# Query to get comments containing "cancer" and top cancer institutes 
q = '''
SELECT "body", "controversiality", "created_utc", "subreddit" 
FROM "AwsDataCatalog"."a05"."a05"
WHERE LOWER("body") LIKE '%cancer%' 
  AND (
      LOWER("body") LIKE '%researcher%' 
      OR LOWER("body") LIKE '%scientist%' 
      OR LOWER("body") LIKE '%physicians%' 
      OR LOWER("body") LIKE '%md anderson cancer center%' 
      OR LOWER("body") LIKE '%memorial sloan kettering cancer center%' 
      OR LOWER("body") LIKE '%msk%' 
      OR LOWER("body") LIKE '%mayo clinic cancer center%' 
      OR LOWER("body") LIKE '%johns hopkins sidney kimmel comprehensive cancer center%' 
      OR LOWER("body") LIKE '%cleveland clinic%' 
      OR LOWER("body") LIKE '%ucla medical center%' 
      OR LOWER("body") LIKE '%massachusetts general hospital cancer center%'
      OR LOWER("body") LIKE '%duke cancer institute%'
      OR LOWER("body") LIKE '%stanford cancer institute%'
      OR LOWER("body") LIKE '%university of california%'
      OR LOWER("body") LIKE '%san francisco medical center%'
      OR LOWER("body") LIKE '%ucsf%'
      OR LOWER("body") LIKE '%northwestern medicine feinberg school of medicine%'
      OR LOWER("body") LIKE '%university of pennsylvania abramson cancer center%'
      OR LOWER("body") LIKE '%roswell park comprehensive cancer center%'
      OR LOWER("body") LIKE '%fred hutchinson cancer research center%'
      OR LOWER("body") LIKE '%trust%'
  ) AND "subreddit" IN ('CrohnsDisease', 'thyroidcancer', 'AskDocs',
'UlcerativeColitis', 'Autoimmune', 'BladderCancer', 'breastcancer', 
'CancerFamilySupport','doihavebreastcancer',
'WomensHealth', 'ProstateCancer', 'cll' ,'Microbiome', 'predental','endometrialcancer',
'cancer','Hashimotos', 'coloncancer','PreCervicalCancer','lymphoma', 'Lymphedema', 'CancerCaregivers',  'braincancer', 'lynchsyndrome',
'nursing','testicularcancer','leukemia','publichealth', 'Health','Fuckcancer','HealthInsurance','BRCA', 'Cancersurvivors','pancreaticcancer', 'skincancer',
'stomachcancer')
'''

# Start the query execution
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# Fetch and load the results
logger.info(response)
df_filtered_comments = download_and_load_query_results(athena_client, response)

# Save the filtered comments to a CSV file
output_file = "Query_A3f.csv"
df_filtered_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

# Preview the filtered DataFrame
print(df_filtered_comments.head())

[2024-11-13 01:09:22,082] p147 {2407217152.py:49} INFO - {'QueryExecutionId': 'f31c27ca-3bd6-4c13-9db8-5f7482588a11', 'ResponseMetadata': {'RequestId': '76370c18-331f-47e9-b7d0-c3f824b1aae4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 01:09:22 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '76370c18-331f-47e9-b7d0-c3f824b1aae4'}, 'RetryAttempts': 0}}
[2024-11-13 01:09:22,083] p147 {272223420.py:4} INFO - download_and_load_query_results, enter
[2024-11-13 01:09:28,455] p147 {272223420.py:17} INFO - Time to complete query: 6.455127000808716s
[2024-11-13 01:09:28,462] p147 {272223420.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-imb59, s3_path=data/a05/f31c27ca-3bd6-4c13-9db8-5f7482588a11.csv, to local file athena_query_results.csv
[2024-11-13 01:09:28,551] p147 {272223420.py:31} INFO - results dataframe shape is (13, 4)
[2024-11-13 01:09:28,556] p147 {2407217152.py:55} INFO - Re

                                                body  controversiality  \
0  I fasted through all my chemo treatments (8 ro...                 0   
1  Cancer has changed everything about my life in...                 0   
2  I was diagnosed with CIN3 after 2 abnormal pap...                 0   
3  Thanks for your reply, and I agree, I apprecia...                 0   
4  What happened to me is the endometrial was dia...                 0   

   created_utc          subreddit  
0   1719839251       breastcancer  
1   1719841803       breastcancer  
2   1719843930  PreCervicalCancer  
3   1719847712                cll  
4   1719849610  endometrialcancer  


In [30]:
# merge the datasets based on the first three hints questions 
import pandas as pd
df_query_a3c = pd.read_csv('Query_A3c.csv')
df_query_a3d = pd.read_csv('Query_A3d.csv')
df_query_a3e = pd.read_csv('Query_A3e.csv')
df_query_a3f = pd.read_csv('Query_A3f.csv')

merged_df = pd.concat([df_query_a3c, df_query_a3d, df_query_a3e, df_query_a3f], ignore_index= True)
merged_df.to_csv('Merged_Data_A3c_A3d_A3e_a3f.csv', index = False)